In [1]:
import osmtogeojson
import requests
import json
import pandas as pd

Get data from Overpass API

In [ ]:
import requests
import pandas as pd

# Define Overpass API endpoint
url = "https://overpass-api.de/api/interpreter"

# Define bounding box coordinates (Replace with your values)
latMin, latMax = 37.49764419371479, 37.56244081620044
lngMin, lngMax = 22.344992459074458, 22.521463853839485

# Overpass QL query to get named streets/ways within the bounding box
query = f"""
[out:json];
(
  way({latMin},{lngMin},{latMax},{lngMax})["highway"]["name"];
);
out geom;
"""

# Make request to Overpass API
response = requests.post(url, data=query)

# Parse JSON response
osm_data = response.json()

# Extract relevant data from OSM JSON response
elements = osm_data.get("elements", [])

# Prepare a list for DataFrame conversion
named_streets_data = []

for element in elements:
    # Extract tags from element
    tags = element.get("tags", {})

    # Ensure the 'name' tag is present
    if 'name' in tags:
        # Handle geometry for 'way' elements (list of coordinates)
        geometry = element.get("geometry", [])
        
        # Add filtered data to the list
        named_streets_data.append([
            element["id"],
            "way",
            tags.get('name', 'Unnamed Street'),  # Default to 'Unnamed Street' if no name
            tags.get('highway', None),
            tags.get('name', None),
            geometry  # Store the list of coordinates for this street
        ])

# Convert to DataFrame
df_named_streets = pd.DataFrame(named_streets_data, columns=["id", "type", "name", "highway", "name_en", "coordinates"])

# Save named streets data to CSV
df_named_streets.to_csv("../DataSets/API_Responses/OSM/named_streets.csv", index=False)

print("Named streets data saved to 'named_streets.csv'")


Named streets data saved to 'named_streets.csv'


In [9]:
import json

# Function to fix the coordinate order for ArcGIS
def fix_coordinates_for_arcgis(geojson_data):
    for feature in geojson_data["features"]:
        # Swap lat and lon for each coordinate in the 'coordinates' list
        feature["geometry"]["coordinates"] = [
            [coord['lon'], coord['lat']] for coord in feature["geometry"]["coordinates"]
        ]
    return geojson_data

# Function to save GeoJSON to file (with correct coordinates)
def save_geojson_to_file(road_name, geojson_data):
    # Define the file name based on the road name
    file_name = "../DataSets/GeoJSON/" + road_name.replace(" ", "_").replace(":", "").replace(";", "") + ".geojson"

    # Fix the coordinate order for ArcGIS compatibility
    geojson_data = fix_coordinates_for_arcgis(geojson_data)

    # Save the corrected GeoJSON data to the file
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(geojson_data, f, ensure_ascii=False, indent=2)

    print(f"GeoJSON data saved to {file_name}")

# Function to get GeoJSON for a specific road by name
def get_geojson_for_road(road_name):
    # Search the dataframe for the road name
    filtered_data = df_named_streets[df_named_streets['name'] == road_name]

    if not filtered_data.empty:
        # Convert the filtered data into GeoJSON format
        geojson = {
            "type": "FeatureCollection",
            "features": []
        }

        # Loop through the rows of filtered data to create features
        for index, row in filtered_data.iterrows():
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": row['coordinates']  # Assuming this is the list of coordinates
                },
                "properties": {
                    "id": row['id'],
                    "name": row['name'],
                    "highway": row['highway']
                }
            }
            geojson["features"].append(feature)
        
        return geojson
    else:
        print(f"Road with name '{road_name}' not found.")
        return None

# Example usage: Get the GeoJSON data for a road
road_name_input = "Αγίου Γεωργίου"  # Replace with your road name
geojson_output = get_geojson_for_road(road_name_input)

# If geojson_output is a valid GeoJSON, save it to a file
if geojson_output:
    save_geojson_to_file(road_name_input, geojson_output)


GeoJSON data saved to ../DataSets/GeoJSON/Αγίου_Γεωργίου.geojson
